In [ ]:
import torch
import deepspeed
from wraptrain import ReadImage,OriginModNetDataLoader,ImageMatteLoader,ModNetImageGenerator,NetTrainer

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torch.nn import Upsample

def _layer_norm(channels, eps=1e-5):
    "Helper function to create a LayerNorm layer for a given number of channels."
    return nn.GroupNorm(1, channels, eps=eps)

class ASPP(nn.Module):
    def __init__(self, in_channels, out_channels, atrous_rates, output_stride=16):
        super(ASPP, self).__init__()
        modules = []
        modules.append(nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 1, bias=False),
            _layer_norm(out_channels),
            nn.ReLU(inplace=True)))

        for rate in atrous_rates:
            modules.append(nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 3, padding=rate, dilation=rate, bias=False),
                _layer_norm(out_channels),
                nn.ReLU(inplace=True)))

        self.global_avg_pool = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_channels, out_channels, 1, bias=False),
            _layer_norm(out_channels),
            nn.ReLU(inplace=True))

        self.convs = nn.ModuleList(modules)

        self.project = nn.Sequential(
            nn.Conv2d((len(modules) + 1) * out_channels, out_channels, 1, bias=False),
            _layer_norm(out_channels),
            nn.ReLU(inplace=True))
        
        self.upsample = nn.Upsample(scale_factor=output_stride, mode='bilinear', align_corners=False)

    def forward(self, x):
        res = [conv(x) for conv in self.convs]
        
        global_feat = self.global_avg_pool(x)
        global_feat = self.upsample(global_feat)
        
        res.append(global_feat)
        res = torch.cat(res, dim=1)
        return self.project(res)


class FeatureFusion(nn.Module):
    def __init__(self, low_channels, mid_channels, high_channels, enhanced_channels, attended_channels, out_channels):
        super(FeatureFusion, self).__init__()
        self.aspp = ASPP(high_channels, out_channels, atrous_rates=[6, 12, 18])
        
        self.conv_low = nn.Sequential(
            nn.Conv2d(low_channels, out_channels, 1, bias=False),
            _layer_norm(out_channels),
            nn.ReLU(inplace=True))
        
        self.conv_mid = nn.Sequential(
            nn.Conv2d(mid_channels, out_channels, 1, bias=False),
            _layer_norm(out_channels),
            nn.ReLU(inplace=True))
        
        self.conv_enhanced = nn.Sequential(
            nn.Conv2d(enhanced_channels, out_channels, 1, bias=False),
            _layer_norm(out_channels),
            nn.ReLU(inplace=True))
        
        self.conv_attended = nn.Sequential(
            nn.Conv2d(attended_channels, out_channels, 1, bias=False),
            _layer_norm(out_channels),
            nn.ReLU(inplace=True))

    def forward(self, low_feat, mid_feat, high_feat, enhanced_feat, attended_feat):
        high_feat = self.aspp(high_feat)
        
        low_feat = self.conv_low(low_feat)
        mid_feat = self.conv_mid(mid_feat)
        enhanced_feat = self.conv_enhanced(enhanced_feat)
        attended_feat = self.conv_attended(attended_feat)
        
        target_size = low_feat.size()[2:]
        high_feat = F.interpolate(high_feat, size=target_size, mode='bilinear', align_corners=True)
        mid_feat = F.interpolate(mid_feat, size=target_size, mode='bilinear', align_corners=True)
        enhanced_feat = F.interpolate(enhanced_feat, size=target_size, mode='bilinear', align_corners=True)
        attended_feat = F.interpolate(attended_feat, size=target_size, mode='bilinear', align_corners=True)
        
        feat = low_feat + mid_feat + high_feat + enhanced_feat + attended_feat
        return feat

# Other modules like FeatureEnhancementModule, PartSegmentationHead, KeyPointHead, MattingHead, and FinteNet
# remain the same in structure but replace BatchNorm2d with LayerNorm using _layer_norm function where applicable.

    
class PartSegmentationHead(nn.Module):
    def __init__(self, in_channels, num_parts):
        super(PartSegmentationHead, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 256, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(256, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, num_parts, kernel_size=1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.conv3(x)
        return x
    
    
class FeatureEnhancementModule(nn.Module):
    def __init__(self, in_channels):
        super(FeatureEnhancementModule, self).__init__()
        self.in_channels=in_channels
        self.channel_attention = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_channels, in_channels // 16, kernel_size=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels // 16, in_channels, kernel_size=1),
            nn.Sigmoid()
        )
        self.spatial_attention = nn.Sequential(
            nn.Conv2d(in_channels, 1, kernel_size=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        #print("feautre ",x.shape)
        #print("channels=",self.in_channels)
        channel_weights = self.channel_attention(x)
        spatial_weights = self.spatial_attention(x)
        #print(channel_weights.shape)
        #print(spatial_weights.shape)
        enhanced_features = x * channel_weights * spatial_weights
        return enhanced_features

class KeyPointHead(nn.Module):
    def __init__(self, in_channels, num_keypoints=13, num_parts=6, upsample_ratio=4):
        super(KeyPointHead, self).__init__()
        self.num_parts = num_parts
        
        self.conv_layers = nn.Sequential(
            nn.Conv2d(in_channels, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, num_keypoints, kernel_size=1)
        )
        #self.channel_matching_conv = nn.Conv2d(adjusted_attention_weights.size()[1], features.size()[1], kernel_size=1)
        self.part_segmentation_head = PartSegmentationHead(in_channels, num_parts)
        #self.feature_enhancement_modules = nn.ModuleList([FeatureEnhancementModule(in_channels) for _ in range(num_parts)])
        #print("in_channels in keypoint",in_channels)
        # 这些错误出现的地方都是通道不匹配
        self.feature_enhancement_modules = nn.ModuleList([FeatureEnhancementModule(in_channels//2) for _ in range(num_parts)])
        self.upsample = nn.Upsample(scale_factor=upsample_ratio, mode='bilinear', align_corners=True)
        self.attention_refine = nn.Sequential(
            nn.Conv2d(num_keypoints, 64, kernel_size=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, num_keypoints, kernel_size=1),
            nn.Sigmoid()
        )
        self.channel_matching_conv = nn.Conv2d(in_channels, in_channels, kernel_size=1)
        self.channel_adjust = nn.Conv2d(num_keypoints, in_channels, kernel_size=1)
        
    def forward(self, x, features):
        keypoint_heatmaps = self.conv_layers(x)
        part_segmentation = self.part_segmentation_head(x)
        
        part_masks = torch.argmax(part_segmentation, dim=1)
        enhanced_features = torch.zeros_like(features)
        
        for i in range(self.num_parts):
            part_mask = (part_masks == i).unsqueeze(1).float()
            upsample_layer = Upsample(scale_factor=2, mode='bilinear', align_corners=False)
            part_mask_upsampled = upsample_layer(part_mask)
            #print(features.shape,part_mask_upsampled.shape)
            part_features = features * part_mask_upsampled
            #print("part future is ",part_features.shape)
            enhanced_part_features = self.feature_enhancement_modules[i](part_features)
            #print("enhanced_part_features is ",enhanced_part_features.shape)
            enhanced_features += enhanced_part_features
        
        upsampled_heatmaps = self.upsample(keypoint_heatmaps)
        attention_weights = self.attention_refine(upsampled_heatmaps)
        adjusted_attention_weights = self.channel_adjust(attention_weights)
        #print("final",features.shape,adjusted_attention_weights.shape)
        # 假设 features 和 adjusted_attention_weights 已经定义并且有上述尺寸

        # 1x1卷积以匹配通道数
        
        channel_matching_conv = nn.Conv2d(adjusted_attention_weights.size()[1], features.size()[1], kernel_size=1)
        # 确定设备
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        #@!!!!!
        # 将卷积层移动到设备上
        channel_matching_conv = channel_matching_conv.to(device)

        # 现在，你可以安全地对位于GPU上的张量进行操作，不会出现设备不匹配的错误

        adjusted_attention_weights_matched = channel_matching_conv(adjusted_attention_weights)

        # 上采样features以匹配空间维度
        features_upsampled = F.interpolate(features, size=adjusted_attention_weights_matched.size()[2:], mode='bilinear', align_corners=True)
    
        #attended_features = features * adjusted_attention_weights
        
        attended_features = features_upsampled * adjusted_attention_weights_matched
        return upsampled_heatmaps, enhanced_features, attended_features


class MattingHead(nn.Module):
    def __init__(self, in_channels, out_channels=1):
        super(MattingHead, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, in_channels // 2, kernel_size=3, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(in_channels // 2)
        self.relu1 = nn.ReLU(inplace=True)
        
        self.conv2 = nn.Conv2d(in_channels // 2, out_channels, kernel_size=3, padding=1, bias=False)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.relu1(self.bn1(self.conv1(x)))
        x = self.sigmoid(self.conv2(x))
        return x

class FinteNet(nn.Module):
    def __init__(self, num_keypoints=13, num_parts=6, pretrained=True):
        super(FinteNet, self).__init__()
        self.mobilenetv3 = models.mobilenet_v3_large(pretrained=pretrained)
        self.features = self.mobilenetv3.features
        
        # Indices for low, mid, and high-level features based on MobileNetV3 structure
        self.low_level_features_index = 3 ##4
        self.mid_level_features_index = 7
        self.high_level_features_index = 14
        
        self.keypoint_head = KeyPointHead(in_channels=160, num_keypoints=num_keypoints, num_parts=num_parts)
        self.feature_fusion = FeatureFusion(low_channels=24, mid_channels=80, high_channels=160, enhanced_channels=80, attended_channels=80, out_channels=256)
        self.matting_head = MattingHead(in_channels=256)  # Assuming fused features have 256 channels
        
    def forward(self, x,reference=True):
        low_level_feat, mid_level_feat, high_level_feat = None, None, None
        
        for i, feature in enumerate(self.features):
            x = feature(x)
            if i == self.low_level_features_index:
                low_level_feat = x
            elif i == self.mid_level_features_index:
                mid_level_feat = x
            elif i == self.high_level_features_index:
                high_level_feat = x

        upsampled_heatmaps, enhanced_features, attended_features = self.keypoint_head(high_level_feat, mid_level_feat)
        fused_features = self.feature_fusion(low_level_feat, mid_level_feat, high_level_feat, enhanced_features, attended_features)
        
        pre_matte = self.matting_head(fused_features)
        
        # Only return the pre_matte and upsampled_heatmaps as specified
        pre_matte = F.interpolate(pre_matte, scale_factor=4, mode='bilinear', align_corners=False)
        return pre_matte, upsampled_heatmaps, pre_matte




In [ ]:
base_path = "/mnt/data/Train/"
fg = base_path+"FG"
matte= base_path+"Alpha"
files = ReadImage(fg,matte).read_same_names()

In [ ]:
all_data = OriginModNetDataLoader(files,[512,512])
t = all_data[0][0]
t = t[None,:,:,:]
print(t.device)
t=t.to("cuda")
print(t.device)




In [ ]:
# 示例使用更新后的模型
model = FinteNet(num_keypoints=13, pretrained=True)

# 获取不同级别的特征和关键点检测结果
#sample,fuse = model(t)

#print("KeyPoints shape:", keypoints.shape,fuse.shape)
#print([i.shape for i in model(t)])

In [ ]:
model = model.to("cuda")

In [ ]:
from torch.utils.data import DataLoader
train_data = OriginModNetDataLoader(files,[512,512])
batch_size = 16 # 例如，每个批次32个样本

train_data = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4)

In [13]:


import torch
import torch.nn as nn
import torch.optim as optim

def train_model(model, dataloader, epochs=1, learning_rate=1e-4, model_save_path='ownmodel.pth'):
    criterion = nn.MSELoss()  # 使用均方误差损失函数
    
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)  # 使用Adam优化器
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  # 检测是否有可用的GPU，若无，则使用CPU
    
    model = model.to(device)  # 确保模型在正确的设备上
    model = torch.nn.DataParallel(model)  # 使用DataParallel来利用多个GPU
    
    model.train()  # 设置模型为训练模式
    
    for epoch in range(epochs):
        running_loss = 0.0
        for batch_idx, (images, trimaps, gt_mattes) in enumerate(dataloader):
            images = images.to(device)  # 将数据移动到当前设备上
            gt_mattes = gt_mattes.to(device)  # 同上
            
            optimizer.zero_grad()  # 清除梯度
            
            pred_matte, _,_ = model(images)  # 正向传播，忽略其他两个输出
            
            loss = criterion(pred_matte, gt_mattes)  # 计算损失
            
            loss.backward()  # 反向传播
            optimizer.step()  # 优化器更新模型参数
            
            running_loss += loss.item()  # 累计损失
            
            if batch_idx % 100 == 99:  # 每100个batch打印一次
                print(f'Epoch: {epoch + 1}, Batch: {batch_idx + 1}, Loss: {running_loss / 100}')
                running_loss = 0.0

    print('Training Complete')
    
    torch.save(model.state_dict(), model_save_path)  # 保存模型
    print(f'Model saved to {model_save_path}')

# 假设`model`和`train_data`已经被定义
model.to(torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))  # 首先将模型转移到正确的设备
train_model(model, train_data, epochs=10, learning_rate=1e-4, model_save_path='ownmodel.pth')



Epoch: 1, Batch: 100, Loss: 0.7105731505155564
Epoch: 2, Batch: 100, Loss: 0.6048527503013611
Epoch: 3, Batch: 100, Loss: 0.5949975144863129
Epoch: 4, Batch: 100, Loss: 0.5883784207701683
Epoch: 5, Batch: 100, Loss: 0.5854550442099571
Epoch: 6, Batch: 100, Loss: 0.586350544989109
Epoch: 7, Batch: 100, Loss: 0.5845371562242508
Epoch: 8, Batch: 100, Loss: 0.578971443772316
Epoch: 9, Batch: 100, Loss: 0.582467348575592
Epoch: 10, Batch: 100, Loss: 0.5786325287818909
Training Complete
Model saved to ownmodel.pth


In [14]:
test = ModNetImageGenerator(files,model,ckpt_path="ownmodel.pth")

RuntimeError: Error(s) in loading state_dict for FinteNet:
	Missing key(s) in state_dict: "mobilenetv3.features.0.0.weight", "mobilenetv3.features.0.1.weight", "mobilenetv3.features.0.1.bias", "mobilenetv3.features.0.1.running_mean", "mobilenetv3.features.0.1.running_var", "mobilenetv3.features.1.block.0.0.weight", "mobilenetv3.features.1.block.0.1.weight", "mobilenetv3.features.1.block.0.1.bias", "mobilenetv3.features.1.block.0.1.running_mean", "mobilenetv3.features.1.block.0.1.running_var", "mobilenetv3.features.1.block.1.0.weight", "mobilenetv3.features.1.block.1.1.weight", "mobilenetv3.features.1.block.1.1.bias", "mobilenetv3.features.1.block.1.1.running_mean", "mobilenetv3.features.1.block.1.1.running_var", "mobilenetv3.features.2.block.0.0.weight", "mobilenetv3.features.2.block.0.1.weight", "mobilenetv3.features.2.block.0.1.bias", "mobilenetv3.features.2.block.0.1.running_mean", "mobilenetv3.features.2.block.0.1.running_var", "mobilenetv3.features.2.block.1.0.weight", "mobilenetv

In [ ]:
pre.shape

#### 